In [1]:
import os
import requests
import json
from tqdm import tqdm
import re
import pandas as pd
from pandas import json_normalize
import time
from dotenv import load_dotenv
load_dotenv()
import openai
from openai import OpenAI
import instructor
import whisper
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.openai_functions import (
    create_structured_output_runnable,
)
from pydantic import BaseModel, Field
from typing import List, Optional
from typing import Sequence
from langchain.pydantic_v1 import BaseModel, Field
from langchain.document_loaders import TextLoader
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [4]:
import backoff
from tqdm import tqdm

@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException, requests.exceptions.HTTPError), max_tries=8)
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
    return response.json()

def fetch_audio_data(base_url, start_page, end_page):
    all_results = []
    for page in tqdm(range(start_page, end_page + 1)):
        url = f"{base_url}?page={page}"
        data = fetch_page(url)
        all_results.extend(data["results"])
        time.sleep(1)
    df = pd.DataFrame.from_records(all_results)
    return df


In [5]:
# Define the URL for the audio endpoint
base_url = "https://www.courtlistener.com/api/rest/v3/audio/"

# Call the function to fetch the data
df = fetch_audio_data(base_url, 1, 10)

100%|██████████| 10/10 [00:15<00:00,  1.52s/it]


In [6]:
df.shape

(200, 24)

In [7]:
df.head()

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,...,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response
0,https://www.courtlistener.com/api/rest/v3/audi...,89173,/audio/89173/appvion-inc-retirement-savin-v-do...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:43.486459-08:00,2023-12-01T14:30:43.486469-08:00,C,,"Appvion, Inc. Retirement Savin v. Douglas Buth",...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,,None,3738.0,True,None,False,0,
1,https://www.courtlistener.com/api/rest/v3/audi...,89172,/audio/89172/denise-evans-v-united-states/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:37.142049-08:00,2023-12-01T14:30:37.142060-08:00,C,,Denise Evans v. United States,...,mp3/2023/12/01/denise_evans_v._united_states_c...,mp3/2023/12/01/denise_evans_v._united_states.mp3,,None,1470.0,True,None,False,0,
2,https://www.courtlistener.com/api/rest/v3/audi...,89171,/audio/89171/randall-artis-v-adrian-santos/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:34.767680-08:00,2023-12-01T14:30:34.767688-08:00,C,,Randall Artis v. Adrian Santos,...,mp3/2023/12/01/randall_artis_v._adrian_santos_...,mp3/2023/12/01/randall_artis_v._adrian_santos.mp3,,None,1652.0,True,None,False,0,
3,https://www.courtlistener.com/api/rest/v3/audi...,89170,/audio/89170/united-states-v-david-hueston/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:31.048186-08:00,2023-12-01T14:30:31.048194-08:00,C,,United States v. David Hueston,...,mp3/2023/12/01/united_states_v._david_hueston_...,mp3/2023/12/01/united_states_v._david_hueston.mp3,,None,1189.0,True,None,False,0,
4,https://www.courtlistener.com/api/rest/v3/audi...,89169,/audio/89169/united-states-v-henry-underwood/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:28.811338-08:00,2023-12-01T14:30:28.811350-08:00,C,,United States v. Henry Underwood,...,mp3/2023/12/01/united_states_v._henry_underwoo...,mp3/2023/12/01/united_states_v._henry_underwoo...,,None,1212.0,True,None,False,0,


In [9]:
df['docket'].head().tolist()

['https://www.courtlistener.com/api/rest/v3/dockets/67709859/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68056321/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68056320/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68056319/',
 'https://www.courtlistener.com/api/rest/v3/dockets/68056318/']

In [10]:
def search_insurance(df, column):
    df['insurance_found'] = df[column].apply(lambda x: 1 if 'insurance' in x.lower() else 0)
    return df

df = search_insurance(df, 'case_name')

In [11]:
print(df[df['insurance_found']==1].shape)
df[df['insurance_found']==1]['case_name'].tolist()

(5, 25)


['Mannie v. Illinois Department of Insurance',
 'Till v. National Health Insurance Co.',
 'Maria Jimenez v. Travelers Commercial Insurance',
 'National Fire Insurance Co. v. Sanchez',
 'Farm Bureau Property & Casualty Insurance Company v. Cleaver']

In [10]:
def add_rows_to_excel(input_df, excel_file):
    """
    Adds rows to an excel file. If the excel file doesn't exist, creates it. 
    If it does exist, only adds the new rows from the input DataFrame. 
    If the data overlaps, only the distinct rows are saved in the final excel file.

    Args:
    input_df (pd.DataFrame): The DataFrame to add to the excel file.
    excel_file (str): The path to the excel file.
    """
    # Convert lists to strings in input_df
    input_df = input_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

    if os.path.exists(excel_file):
        # Load the existing data
        existing_df = pd.read_excel(excel_file)

        # Convert lists to strings in existing_df
        existing_df = existing_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

        # Concatenate the existing data with the new data
        combined_df = pd.concat([existing_df, input_df])

        # Remove duplicates
        final_df = combined_df.drop_duplicates()

        # Print the total number of rows and how many new rows were added
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df) - len(existing_df)}")
    else:
        final_df = input_df
        # Print the total number of rows as the excel file didn't exist before
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df)}")

    # Write the data to the excel file
    final_df.to_excel(excel_file, index=False)

In [11]:
add_rows_to_excel(df, "court_listener_oral_argument_listing.xlsx")

Total number of rows: 3988
Number of new rows added: 200


In [13]:
insurance_df = df[df["insurance_found"] != 0]

i = 0

name = insurance_df.iloc[i]["case_name"]
url = insurance_df.iloc[i]["download_url"]
print(name)
print(url)

Mannie v. Illinois Department of Insurance
https://ilcourtsaudio.blob.core.windows.net/antilles-resources/resources/dc00b730-4ff0-4764-96dc-6054f5e4070c/113023_3-22-0312.mp3


In [14]:
def download_mp3(url, file_path):
    """
    Downloads an MP3 file from the given URL and saves it to the specified file path.

    Args:
    url (str): The URL of the MP3 file.
    file_path (str): The local path where the file should be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)

        # Raise an exception if the request was unsuccessful
        response.raise_for_status()

        # Open a file in binary write mode
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                # Write the content chunk by chunk to the file
                file.write(chunk)

        print(f"File downloaded and saved as {file_path}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [15]:
download_mp3(url, f"./data/audio/{name}.mp3")

File downloaded and saved as ./data/audio/Mannie v. Illinois Department of Insurance.mp3


In [17]:
%%time
model = whisper.load_model("base")
result = model.transcribe(f"./data/audio/{name}.mp3")
Markdown(f"{result['text']}")

CPU times: total: 15min 9s
Wall time: 1min 57s


 We'll look, please call the next case. 322-0312, Kenneth Armani, senior appellant versus Illinois Department of Insurance, Martin Grovy, Helen Kim, and Patrick Riley, Appalise. Thank you. Mr. Manny, you may proceed. Thank you. I'll start the wagon before this point will be about Mary's team. Practice is not a mechanic. I'll start the practice in the maintenance. But Mr. Manny, unlike the case before, we're having just a little hard time understanding you. If you may be getting closer to the screen or the mic, might help. OK, about now. Any better? Little. I'll try to speak real loud. OK, OK. We fine. Thank you. OK. I'm this court is wrong with it. I am not an attorney. And because of the actions of the defendant, I'm an indigent and I cannot afford it. That's why I have a fee waiver. This case was filed as a result of the Office of the Executiveist General. And with a preceded court, we're forced to defend it to stick to the matter that's all in this case only. Plenty of thousands of checks, all blacksmiths, preferton, this case, gotments D1 through D10 that was extracted from the original brief. The Tendentomian Exile was complaint with the Illinois Department of Insurance against field marketing organization. Health care submission team, health care submission team, supervisor manager and time management change. Now owned by all state. Field marketing organization supervises and oversee everything that agents do. Agents cannot work directly within the health companies. They must go through a field marketing organization. And then the Illinois Department of Insurance is the governing body. Defended Illinois Department of Insurance responded to my complaint against the defendants. And was sent a copy of the Illinois General Summit's thank you showing that all the policies that was written by health care solutions team and health care submission team, supervisor, were fraudulent policies. The Illinois Department of Insurance decided not to do anything and they sent me a letter December 3rd, 2014. And that's document P1 that you have. And it says that they found nothing wrong with the actions and the fraudulent insurance policies that was written on the plaintiff. The plaintiff filed a complaint with the EEOC Equal Employment Opportunity Commission and the Illinois Department of Human Rights because the Department of Insurance chose to not do anything in response to all the crimes that was committed against the plaintiff. When the plaintiff filed that complaint, the Illinois Department of Insurance retaliated against the plaintiff and charged them with check fraud by using a photo copy of one of the checks that was set in the complaint. The Illinois Department of Insurance filed a plaintiff guilty of check fraud which violates all ordinances and then they revoked the plaintiff's produces license that he had for over 20 years. If you google the plaintiff now, he comes up as a check board and that destroyed the life and integrity of the plaintiff. That's the three waivers. It also caused the plaintiff that was contacted by the Obama administration to assist with health care reform and wrote in a dendum and all Congress rejected and would not review any of that. The dendum written by the plaintiff would save this nation over $100 billion that totally paid for Obamacare. And no member of Congress would address that a dendum because of the actions of the defendant, Illinois Department of Insurance. The Human Rights and Equal Opportunity Commission questioned the legal authority of the Illinois Department of Insurance as it relates to check fraud. And office of the executive and executive general got involved. On September 1st, 2020, office of the executive general ordered the Illinois Department of Insurance to conduct a rehearing and that rehearing took place on September 1st. During the hearing, the Illinois Department of Insurance was asked to provide a complaint from anyone saying that the plaintiff had forged a check or altered a check. Illinois Department of Insurance provided no complaint because no complaint exists. And if you look at Illinois law, B5, on the documents provided, it clearly states that only prosecutors have the authority to prosecute individuals and entities which have fraud. So the defendant had no complaint and no authority to make any charges of set fraud according to Illinois law. No third parties can try to make charges. During the rehearing, the Illinois Department of Insurance was asked to provide a original document, the original check and pay attention to the author of the report. Illinois Department of Insurance could not provide any document because it didn't exist. Check's refund checks were sent to the address at Peter Benson, supervisor manager for health care solutions team. All refunds were sent there because of the address on the check had the jolly address of supervisor manager Peter Benson. And then those check were returned to Times of Sharon where they boarded them out. During the hearing, the Illinois Department of Insurance was asked to provide the five elements required by state of Illinois to prove check fraud. That's a D7, five elements necessary to prove check fraud. The Illinois Department of Insurance failed to provide any document, anything to prove any of the five elements required to prove check fraud. And lastly during the rehearing, the Illinois Department of Insurance was required was asked to provide records of any other Illinois insurance producer that the Illinois Department of Insurance had prosecuted for offering photo copies of refund checks. Illinois Department of Insurance could provide no other cases where anyone was prosecuted by Illinois Department of Insurance for check fraud using photo copies. And without any complaint, saying that check fraud was committed. On February 16, 2016, the Illinois Department of Insurance and the Fair Decision on the rehearing and found the plaintiff again liable and guilty of check fraud. And VHOs were the targeting check altering and found me liable without any authority, without any complaint, and without any original documents violating all Illinois laws. And they provided no records of anyone else being prosecuted ever by the Illinois Department of Insurance for check fraud when there was no check using photo copies that were sent to several different locations. In July of 2021, plaintiff found lawsuit 2021, L-00136 for charging Illinois Department of Insurance with malicious and false prosecution in DuPage County where the plaintiff was residing in crime and requested a free trial. The lower court refused to hear this case and did not allow jury and appears to be selected for this case. The plaintiff is asking this court to disbake that order to dismiss re-instate the case, force the defendant, Illinois Department of Insurance, re-instate the plaintiff's Illinois-Cupus license without lapse and award damages to the plaintiff from the defendant. For these false and illegal actions that defendant did completely destroy the life and reputation of the plaintiff. This court has copies of the document and documents provided to substantiate all claims by the plaintiff that was prepared and sent with this or argument so that the court can review them. Thank you. Any questions from the court? I do not. Thank you, Mr. Manny. You will have time and reply. Miss Omera? Yes, thank you. You may respond. Thank you. Good afternoon, Your Honors. May I please the court? My name is Assistant Attorney General Laura Omera. I'm a student at the Illinois Department of Insurance. Mr. Manny's complaint stems from the final administrative decision of the department to suspend his license based on its findings that he engaged in dishonest practices. Manny appealed that decision to the circuit court and to this court in 2018, both of which upheld the department's decision holding that there was ample support in the record for the department's decision. Since that time, Manny has filed several lawsuits against the department in which he continues to challenge the department's final administrative decision. His complaint was properly dismissed by the circuit court for several reasons. First, the circuit court lacked jurisdiction over this action because it was an improper collateral attack on a final administrative decision of the department. As I'm sure this court is aware, the circuit court has jurisdiction to review agency decisions only as provided by law and the administrative review law, which applies here under the Illinois insurance code, provides that a party may see review of an administrative decision only within the time and the manner prescribed by statute. Manny's initial administrative review action, which he properly brought pursuant to the administrative review law, was the only method for challenging the department's decision in 2015. Once that action concluded, any subsequent litigation such as Manny's complaint and due page county, is an improper collateral attack on the agency's decisions. The complaint is also barred by statute of caught out along the same lines. As we explained in our brief, the court has already addressed Manny's claims and the administrative review action. These are the same issues, the same parties, and Manny's complaint in this case is an attempt to re-litigate claims that have already been decided. Several immunity would also apply here to bar Mr. Manny's claims because the department has a state agency and he has not alleged any facts and has complained that would support the application of any exception to that doctrine. And finally, Mr. Manny has failed to state a claim because he has not alleged any specific facts and is complained that would support a claim of malicious prosecution or slanderer. I would like to respond to Mr. Manny's arguments about check fraud. He continues to argue that he was charged convicted found guilty of check fraud. But that is simply not the case. The department never charged him with check fraud or any other crime. The department found him guilty or found that he engaged in dishonest and untrustworthy practices under the Illinois insurance code, which the department does have the authority to suspend one's license. Under that statute. So these were entirely civil proceedings and his reference to the criminal code is misplaced. And I also wanted to touch on his comments about the 2020 re-hearing that was ordered by the inspector general. I'm not sure I don't, I'm the inspector general was not involved in this. As far as I'm aware, but in 2020 or in 2019, Mr. Manny applied for his license renewal. That application was denied because he there was evidence that he sold insurance policies during his 18 month suspension, as well as evidence that he responded to a few questions on the application. So it wasn't a re-hearing in 2020. It was just a hearing that he requested on the denial of his application. So that was an entirely separate proceeding and he seems to be blurring the lines a little bit between those two in 2021, 2020 again, no one charged him with check fraud. He was found to be again held responsible for engaging in a different set of dishonest and untrustworthy conduct. And so for those reasons, his application was denied in February of 2021, I believe. So I just want to know is it is it not true that yes, but that one of those practices was the alteration of a check set. Well, that was the basis for the 2015 final administrative decision was the altering of the check. That was not an issue at all with the 2020 proceedings. They were separate administrative proceedings. And Mr. Manny did not appeal the 2021 final administrative decision of the department. So they were two separate proceedings in 2015. Yes, he was found that it was found that he altered the check and that decision was appalled by this court and the decision and anyone. I think that's all I have on most of the court has any further questions. We stand on a brief and ask the court to affirm the dismissal of this complaint. Any questions from the bench. I'm assuming I answered my question. I do not. Thank you. Mr. Mr. Manny, you may reply. I'm not an officer of the executive panel does have a quality they do. And during the second hearing, and the second hearing was ordered September, the place September 1st, 2020. Now, he's saying that the altering of the check didn't take place. It's a letter you have back in the B8, dated January 17, 2020, from the Elemmon Department of Insurance. And it says right in circle there due to altering a premium refund check. That's what I was charged with. And they have no authority. They have no complaint saying that that took place. It was all fabricated by the Elemmon Department of Insurance. Totally fabricated. They have no complaint. That was during the during the rehearing. All those questions was asked of the Department of Insurance. They asked to provide a complaint. They couldn't do that. They asked to provide. They provide there are 40 to make charges, which are Illinois courts. They can't make charges. They have third party. They didn't provide that. It was at the past providing a genuine original document of the check. They couldn't do that. And they couldn't come up with one of the five elements necessary. That was asked during the rehearing on September 1, 2020. That was ordered by the office of the executive committee. And this letter, dated January 17, 2020, says just that due to altering a refund check. So that's what they did. They definitely charged me with that. They have no authority. They have no complaint. They have no documents. They fabricated this entire entire charge in retaliation for my charging them with the Equal Opportunity Commission and the Department of Human Rights. They have no authority according to Illinois law to make charges of check fraud altering checks. None. And they say internal investigation. There's no complaint to stimulate that. Someone had to say a check was altered or something by me. Didn't happen. The check in question was mailed to Peter Benson in Juliette and returned two times a shunner where they voted to check out. When the Illinois Department of Insurance saw that in my complaint. And they did nothing to these organizations for what they did for me. And sent a first letter saying we don't find anything wrong with them. Everything your account. Right in front of fraudulent insurance policies on you. I filed a complaint with the ELC and they did not like that. So they fabricated everything. They don't have one document again with a complaint against me. They don't have one document showing that they have an authority to make check fraud practice. She said they didn't do that. And you see right in this letter right here. It's circled due to altering a refund check. Right there on that letter, they did. And every document that I sent to you also has that exact information. That was why the rehearing was ordered by the office of the general that does have authority over the Illinois Department of Insurance. That's why I sent a copy of the office of the executive inspector and all of their authority. I sent that with this argument. So the facts are they fabricated everything. No complaint exists, no authority exists, no original document exists. And they didn't suspend to one element necessary to prove. Get all to your. Thank you. Thank you, Mr. Manny. Any questions from the bench for Mr. Manny? Welcome. The only question I had is as I did in the last case, I asked this court force to defend it to stick to this case. This case was filed in 2021. And in every case, everybody wants to talk about what happened before. When once again, no facts were provided and they prevail. This case was ordered by the executive office of the executive inspector. And this is the only case that should be. Mr. Manny, do you have any exhibits that verify that this was ordered by the office of the. Yes, general. There were documents in the. In your outline of your oral breed. And in outline of the. In both. And also, but a copy of the letter from the office of the executive. I think it's made it. 2021 saying that they've made their decision. We suggest you hire an attorney. Is made it. May fifth from the office of the executive. We suggest you hire an attorney because once again. They haven't shown any complaint. Any recording. They haven't shown any original documents. And they haven't. Said anything about the five elements necessary. And the letter dated me. May 20. May, I think it's. Is it was. Telling me. The higher the attorney. Is it says. Никit. Is a thing that. Once again, they didn't. One complaint. they have the 40 to make charges of cash cards, which is limited to the attorney general and state attorney. So they had no complaint, no authority, they didn't have one original document. And they say, well, you violated all these things. What rules are I violating? I didn't do any of that. And how can that happen? And like I said, it was all fabricated because they didn't like the complaint that I filed with the EEOC and the Department of Human Rights. That's why the Office of the Executive Committee sent me this saying, hey, we are just defined as an attorney, which I can't add up. Okay. Mr. Mani, Ms. O'Mara, thank you for your arguments in this matter this afternoon. It will be taken under advisement in a written disposition sale issue. The clerk of our court at this time will escort you out of our remote courtroom and have a good afternoon. Thank you.

In [30]:
def save_dict_to_txt_file(data_dict, file_name):
    """
    Saves the 'text' value from a dictionary to a .txt file.

    Args:
    data_dict (dict): The dictionary containing the data.
    file_name (str): The name of the .txt file to save the data to.
    """
    # Get the 'text' value from the dictionary
    text = data_dict.get('text', '')

    # Clean the file name by removing spaces, punctuation, and special characters
    file_name = re.sub(r'\W+', '', file_name)

    # Create 'transcriptions' directory if it doesn't exist
    if not os.path.exists('data/transcriptions'):
        os.makedirs('data/transcriptions')

    # Open the file in write mode
    with open(f"data/transcriptions/{file_name}.txt", 'w', encoding='utf-8') as file:
        # Write the text to the file
        file.write(text)

    print(f"Data saved to data/transcriptions/{file_name}.txt")


In [2]:
def read_text_from_txt_file(file_name):
    """
    Reads the text from a .txt file.

    Args:
    file_name (str): The name of the .txt file to read the data from.
    
    Returns:
    str: The text read from the file.
    """
    # Check if the file exists
    if not os.path.exists(f"data/transcriptions/{file_name}.txt"):
        print(f"File data/transcriptions/{file_name}.txt does not exist.")
        return None

    # Open the file in read mode
    with open(f"data/transcriptions/{file_name}.txt", 'r', encoding='utf-8') as file:
        # Read the text from the file
        text = file.read()

    return text

In [32]:
save_dict_to_txt_file(result, name)

Data saved to data/transcriptions/MannievIllinoisDepartmentofInsurance.txt


In [7]:
text = read_text_from_txt_file("MannievIllinoisDepartmentofInsurance")

In [42]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [43]:
num_tokens_from_string(text, "cl100k_base")

3573

In [9]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=15000,chunk_overlap=0,separator="\n")

def run_map_reduce(input_doc, MAP_PROMPT, REDUCE_PROMPT):

    llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.1)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.1)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="text_summaries")

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=15000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="text",
        # Return the results of the map steps in the output
        return_intermediate_steps=False)

    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)

    return map_reduce.run(input_text=input_doc)

In [10]:
map_template_string = """Context: {text}
As an expert legal analyst, review the Context which is part of an oral argument session and do three things:

1. Identify the key facts of the case.
2. Summarize plaintiff counsel's main arguments.
3. Summarize rebuttals to plaintiff counsel's arguments.
"""

reduce_template_string = template = """Context: {text_summaries}
As an experienced legal analyst, use the context to:

Compose a case brief using only the Context.
Working step by step, organize the Context into a well-structured
outline with the following sections:

Facts of Loss: a concise sumary of facts and events relevant to this legal case.
Issue: [the facts of the case being argued]
Analysis: arguments
   ["Argument 1": key points from a distinct argument,
    "Rebuttal 1": key points from the rebuttal to argument 1,
    "Argument 2": key points from a distinct argument,
    "Rebuttal 2": key points from the rebuttal to argument 2,
    "Argument X": ...,
    "Rebuttal X": ...]
Takeaway: the most important argument assertion and its corresponding rebuttal.
TODO: optional list of knowledge gaps, or logical flaws in any of the arguments or their rebuttals.
"""
MAP_PROMPT = PromptTemplate(input_variables=["text"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["text_summaries"], template=reduce_template_string)

In [11]:
result = run_map_reduce(text, MAP_PROMPT, REDUCE_PROMPT)

In [12]:
display(Markdown(f"{result}"))

### Case Brief

**Title:** Kenneth Armani v. Illinois Department of Insurance et al.

**Facts of Loss:**
Kenneth Armani, the appellant, is self-represented in a legal dispute against the Illinois Department of Insurance and associated parties. Armani, who held a producer's license for over 20 years, claims that the Department wrongfully accused him of check fraud and revoked his license. He alleges that this revocation was retaliatory, following his complaints to the EEOC and the Illinois Department of Human Rights. Armani argues that the Department lacked authority and evidence to charge him with check fraud and that their actions have severely damaged his life and reputation. He seeks to reverse the dismissal of his case, reinstate his license, and obtain damages.

**Issue:**
Whether the Illinois Department of Insurance wrongfully revoked Kenneth Armani's producer's license on the grounds of check fraud without proper authority or evidence, and whether the revocation was retaliatory in nature.

**Analysis:**

**Argument 1:** Armani contends that the Department of Insurance had no legal basis to charge him with check fraud, as they provided no original documents or evidence, and that the charges were fabricated.

**Rebuttal 1:** Assistant Attorney General Laura O'Mara argues that the Department never criminally charged Armani with check fraud but found him engaged in dishonest practices under the civil provisions of the Illinois insurance code.

**Argument 2:** Armani asserts that the Department's actions were in retaliation for his prior complaints to the EEOC and the Illinois Department of Human Rights.

**Rebuttal 2:** O'Mara does not directly address the retaliation claim but focuses on the legitimacy of the administrative decisions and the separate proceedings related to Armani's license renewal.

**Argument 3:** Armani points to the Office of the Executive Inspector General's order for a rehearing as evidence that the Department acted without proper authority.

**Rebuttal 3:** O'Mara maintains that the circuit court lacked jurisdiction over Armani's action as it was an improper collateral attack on a final administrative decision.

**Takeaway:**
The most critical argument is Armani's claim that the Department had no legal basis to charge him with check fraud, which is directly countered by O'Mara's assertion that the Department found Armani engaged in dishonest practices under the Illinois insurance code, not criminal check fraud.

**TODO:**
- Verify the details of the Office of the Executive Inspector General's order for a rehearing and its implications on the case.
- Clarify whether there is any evidence supporting Armani's retaliation claims.
- Assess the validity of the jurisdiction argument presented by O'Mara and whether Armani's action is indeed an improper collateral attack on a final administrative decision.
- Examine the specifics of the 2020 proceedings related to Armani's license renewal and the evidence of selling insurance policies during his suspension.
- Determine the applicability of sovereign immunity in this case and whether Armani's claims for malicious prosecution or slander are barred by it.

In [13]:
class CourtroomParty(BaseModel):
    """Identifying information about a person."""
    
    person: str = Field(..., description="The name of the person speaking")
    role: Optional[str] = Field(None, description="The role of the person speaking")
    background: Optional[str] = Field(None, description="A concise summary about the person")
    

class OralArgument(BaseModel):
    """Identifying information about a particular argument."""
    
    topic: str = Field(..., description="A concise topic title for the argument")
    side: Optional[str] = Field(None, description="The side the argument favors, plaintiff or defense")
    person: Optional[str] = Field(None, description="The name of the person presenting the argument")
    summary: Optional[str] = Field(None, description="A concise summary of key points")
    

class OralDocket(BaseModel):
    """Identifying information in a text."""

    parties: Sequence[CourtroomParty] = Field(..., description="The information of interest in the text")
    arguments: Sequence[OralArgument] = Field(..., description="The information of interest in the text")

In [50]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3040,
    chunk_overlap  = 0,
    is_separator_regex = False,
)

In [15]:
loader = TextLoader("data/transcriptions/MannievIllinoisDepartmentofInsurance.txt", encoding='utf-8')
doc = loader.load()

In [16]:
docs = text_splitter.split_documents(doc)

In [17]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {input}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)

In [18]:
runnable = create_structured_output_runnable(OralDocket, llm, prompt)

In [19]:
res_doc = runnable.invoke({"input": doc})

In [20]:
print(res_doc)

parties=[CourtroomParty(person='Kenneth Armani', role='Senior Appellant', background="Indigent, cannot afford an attorney, has a fee waiver, filed a case as a result of actions by the Office of the Executive Inspector General, claims to have been retaliated against by the Illinois Department of Insurance with false charges of check fraud, resulting in the revocation of his producer's license."), CourtroomParty(person='Martin Grovy', role='Appellee', background=None), CourtroomParty(person='Helen Kim', role='Appellee', background=None), CourtroomParty(person='Patrick Riley', role='Appellee', background=None), CourtroomParty(person='Laura Omera', role='Assistant Attorney General', background="Representing the Illinois Department of Insurance, argues that Kenneth Armani's complaint was properly dismissed and that the department's actions were justified.")] arguments=[OralArgument(topic='Fee Waiver and Indigence', side='Plaintiff', person='Kenneth Armani', summary='Kenneth Armani claims in

In [146]:
class CourtListenerExtractor:
    def __init__(self, database_path, base_url="https://www.courtlistener.com/api/rest/v3/audio/"):
        """
        Initializes the CourtListenerExtractor with the path to the database file.

        Args:
        database_path (str): The path to the Excel file database.
        """
        self.database_path = database_path
        self.base_url=base_url

    def fetch_records(self, start_page, end_page):
        """
        Fetches records from Court Listener based on a page range.
        
        Args:
        start_page (int): court listener web search start
        end_page (int): court listener web search start

        Returns:
        pd.DataFrame: A DataFrame containing the fetched records.
        """
        all_results = []
        for page in tqdm(range(start_page, end_page + 1)):
            url = f"{self.base_url}?page={page}"
            data = self.fetch_page_(url)
            all_results.extend(data["results"])
            time.sleep(1)
        df = pd.DataFrame.from_records(all_results)
        return df

    def add_rows_to_excel(self, input_df):
        # Convert lists to strings in input_df
        input_df = input_df.applymap(lambda x: str(x) if isinstance(x, list) else x)
        
        existing_df = pd.DataFrame()
        if os.path.exists(self.database_path):
            # Load the existing data
            existing_df = pd.read_excel(self.database_path)

            # Convert lists to strings in existing_df
            existing_df = existing_df.applymap(lambda x: str(x) if isinstance(x, list) else x)

        # Concatenate the existing data with the new data
        combined_df = pd.concat([existing_df, input_df])

        # Remove the 'unique_index' column before removing duplicates
        combined_df = combined_df.drop(columns=['unique_index'])

        # Remove duplicates
        final_df = combined_df.drop_duplicates()

        # Reassign the 'unique_index' column
        final_df['unique_index'] = range(1, len(final_df) + 1)

        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of new rows added: {len(final_df) - len(existing_df)}")

        # Write the data to the Excel file
        final_df.to_excel(self.database_path, index=False)

    def update_database(self, new_records):
        """
        Updates the database with new records.

        Args:
        new_records (pd.DataFrame): New records to be added to the database.
        """
        self.add_rows_to_excel(new_records)
        
    @backoff.on_exception(backoff.expo, (requests.exceptions.RequestException, requests.exceptions.HTTPError), max_tries=8)
    def fetch_page_(self, url):
        response = requests.get(url)
        response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
        return response.json()
    
    def find_subset_with_keyword(self, keyword="insurance"):
        """
        Finds a subset of records containing a specific keyword.

        Args:
        keyword (str, optional): The keyword to search for. Defaults to "insurance".

        Returns:
        pd.DataFrame: A DataFrame containing the subset of records with the keyword.
        """
        if not os.path.exists(self.database_path):
            raise FileNotFoundError(f"Database file {self.database_path} not found.")

        # Load the existing data
        df = pd.read_excel(self.database_path)

        # Finding the subset with the keyword
        subset_df = df[df.apply(lambda row: row.astype(str).str.contains(keyword).any(), axis=1)]
        print(f"Records with keyword: {keyword}: {len(subset_df)}")
        
        return subset_df

# Example usage:
# extractor = CourtListenerExtractor("path/to/database.xlsx")
# new_records = extractor.fetch_records(1, 11)
# extractor.update_database(new_records)

In [129]:
extractor = CourtListenerExtractor("database.xlsx")

In [130]:
new_records = extractor.fetch_records(1, 6)

100%|██████████| 6/6 [00:08<00:00,  1.48s/it]


In [131]:
new_records.head(2)

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response
0,https://www.courtlistener.com/api/rest/v3/audi...,89173,/audio/89173/appvion-inc-retirement-savin-v-do...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:43.486459-08:00,2023-12-01T14:30:43.486469-08:00,C,,"Appvion, Inc. Retirement Savin v. Douglas Buth",,,f2922a7f6a7085a3a6047ab6cd64e73d5e2d2180,http://media.ca7.uscourts.gov/sound/external/k...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,mp3/2023/12/01/appvion_inc._retirement_savin_v...,,None,3738.0,True,None,False,0,
1,https://www.courtlistener.com/api/rest/v3/audi...,89172,/audio/89172/denise-evans-v-united-states/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-12-01T14:30:37.142049-08:00,2023-12-01T14:30:37.142060-08:00,C,,Denise Evans v. United States,,,37fa42c525b70a19c7ededd868cb9219d0bac5c8,http://media.ca7.uscourts.gov/sound/external/k...,mp3/2023/12/01/denise_evans_v._united_states_c...,mp3/2023/12/01/denise_evans_v._united_states.mp3,,None,1470.0,True,None,False,0,


In [132]:
extractor.update_database(new_records)

Total number of rows: 340
Number of new rows added: 120


In [145]:
class AudioTranscriber:
    def __init__(self, download_path, text_output_path, model_name='base'):
        self.download_path = download_path
        self.text_output_path = text_output_path
        self.transcription_model = whisper.load_model(model_name)
        self._create_directory_if_not_exists(download_path)
        self._create_directory_if_not_exists(text_output_path)

    def _create_directory_if_not_exists(self, path):
        if not os.path.exists(path):
            os.makedirs(path)

    def download_mp3(self, url, file_name, unique_index):
        file_path = os.path.join(self.download_path, f"{file_name}_{unique_index}.mp3")
        if os.path.exists(file_path):
            print(f"File {file_name}.mp3 already exists.")
            return file_path

        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()

            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

            print(f"File downloaded and saved as {file_path}")
            return file_path
        except requests.RequestException as e:
            print(f"An error occurred: {e}")

    def transcribe_audio(self, file_path):
        result = self.transcription_model.transcribe(file_path)
        return result['text']
    
    def save_transcription(self, text, file_name, unique_index):
        text_file_path = os.path.join(self.text_output_path, f"{file_name}_{unique_index}.txt")
        with open(text_file_path, 'w') as file:
            file.write(text)
        print(f"Transcription saved as {text_file_path}")

    def read_transcription(self, file_name, unique_index):
        text_file_path = os.path.join(self.text_output_path, f"{file_name}_{unique_index}.txt")
        if not os.path.exists(text_file_path):
            print(f"No transcription file found for {file_name}")
            return None
        with open(text_file_path, 'r') as file:
            return file.read()

    def process_batch(self, df):
        for _, row in df.iterrows():
            url = row['download_url']
            name = re.sub(r'\W+', '', row['case_name'])
            unique_index = row['unique_index']
            file_path = self.download_mp3(url, name, unique_index)
            if file_path:
                transcription = self.transcribe_audio(file_path)
                self.save_transcription(transcription, name, unique_index)

# Example usage:
# audio_transcriber = AudioTranscriber('path/to/download', 'path/to/text_output')
# extractor = CourtListenerExtractor("database.xlsx")
# df = extractor.fetch_records(1, 11)
# audio_transcriber.process_batch(df)

In [137]:
audio_transcriber = AudioTranscriber('audio', 'transcriptions')

In [138]:
extractor = CourtListenerExtractor("database.xlsx")

In [ ]:
df = extractor.fetch_records(1, 11)

In [139]:
insurance_cases = extractor.find_subset_with_keyword()

In [141]:
insurance_cases.head(2)

,resource_uri,id,absolute_url,panel,docket,date_created,date_modified,source,case_name_short,case_name,case_name_full,judges,sha1,download_url,local_path_mp3,local_path_original_file,filepath_ia,ia_upload_failure_count,duration,processing_complete,date_blocked,blocked,stt_status,stt_google_response,unique_index
16,https://www.courtlistener.com/api/rest/v3/audi...,89157,/audio/89157/mannie-v-illinois-department-of-i...,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-30T14:54:57.601458-08:00,2023-12-01T01:38:54.934186-08:00,C,Mannie,Mannie v. Illinois Department of Insurance,NaN,NaN,efc0ffa7dcce6495ee918087d7d986f520f89168,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/11/30/mannie_v._illinois_department_o...,mp3/2023/11/30/mannie_v._illinois_department_o...,https://archive.org/download/gov.uscourts.illa...,NaN,1212.0,True,NaN,False,0,NaN,17
18,https://www.courtlistener.com/api/rest/v3/audi...,89155,/audio/89155/till-v-national-health-insurance-co/,[],https://www.courtlistener.com/api/rest/v3/dock...,2023-11-30T14:54:52.407621-08:00,2023-12-01T02:00:48.933627-08:00,C,Till,Till v. National Health Insurance Co.,NaN,NaN,e4cceca79c20f86832d1eb77c122bfceb6597a77,https://ilcourtsaudio.blob.core.windows.net/an...,mp3/2023/11/30/till_v._national_health_insuran...,mp3/2023/11/30/till_v._national_health_insuran...,https://archive.org/download/gov.uscourts.illa...,NaN,2153.0,True,NaN,False,0,NaN,19


In [142]:
%%time
audio_transcriber.process_batch(insurance_cases.head(2))

File downloaded and saved as audio\MannievIllinoisDepartmentofInsurance.mp3
Transcription saved as transcriptions\MannievIllinoisDepartmentofInsurance.txt
File downloaded and saved as audio\TillvNationalHealthInsuranceCo.mp3
Transcription saved as transcriptions\TillvNationalHealthInsuranceCo.txt
CPU times: total: 41min 45s
Wall time: 5min 23s


In [150]:
class OralDocketExtractor:
    def __init__(self, model_name="gpt-4-1106-preview"):
        self.model = ChatOpenAI(model=model_name, temperature=0.1)

    def process_document(self, document_path: str) -> OralDocket:
        loader = TextLoader(document_path)
        doc = loader.load()
            
        prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a world class algorithm for extracting information in structured formats.",
            ),
            (
                "human",
                "Use the given format to extract information from the following input: {input}",
            ),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

        runnable = create_structured_output_runnable(OralDocket, self.model, prompt)
        result = runnable.invoke({"input": doc})
        
        # Extract unique_index from the filename
        filename = os.path.basename(document_path)
        unique_index = filename.split("_")[-1].split(".")[0]
        
        return result, unique_index
    
    def save_to_excel(self, data: List[OralDocket], file_name: str, unique_index: int):
        data_dicts = [instance.dict() for instance in data]

        # Save 'parties' and 'arguments' to separate sheets
        for key in ['parties', 'arguments']:
            df_new = pd.concat([pd.json_normalize(data_dict[key]) for data_dict in data_dicts], ignore_index=True)
            df_new['unique_index'] = unique_index  # Add the unique_index column
            self.write_to_excel(df_new, key, file_name)

    @staticmethod
    def write_to_excel(df_new, sheet_name, file_name='court_whisperer_data.xlsx'):
        if os.path.exists(file_name):
            try:
                # Attempt to read the existing sheet
                df_old = pd.read_excel(file_name, sheet_name=sheet_name)
                df = pd.concat([df_old, df_new]).drop_duplicates()
            except ValueError:
                # If the sheet does not exist, use the new dataframe as is
                df = df_new

            # Open the existing workbook using 'a' mode (append)
            with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                # If the sheet exists, it will be overwritten
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # Create a new file if it doesn't exist
            df_new.to_excel(file_name, sheet_name=sheet_name, index=False)


In [147]:
# Example usage:
audio_transcriber = AudioTranscriber('audio', 'transcriptions')
extractor = CourtListenerExtractor("database.xlsx")
df = extractor.fetch_records(1, 21)
insurance_cases = extractor.find_subset_with_keyword()
audio_transcriber.process_batch(insurance_cases.tail(2))

100%|██████████| 21/21 [00:31<00:00,  1.50s/it]


Records with keyword: insurance: 11
File downloaded and saved as audio\MariaJimenezvTravelersCommercialInsurance_249.mp3
Transcription saved as transcriptions\MariaJimenezvTravelersCommercialInsurance_249.txt
File downloaded and saved as audio\NationalFireInsuranceCovSanchez_282.mp3
Transcription saved as transcriptions\NationalFireInsuranceCovSanchez_282.txt


In [152]:
llm_processor = OralDocketExtractor()

In [153]:
text_files = [
    "MariaJimenezvTravelersCommercialInsurance_249.txt",
    "NationalFireInsuranceCovSanchez_282.txt"
]

for text_file in text_files:
    result, unique_index = llm_processor.process_document(f"transcriptions/{text_file}")
    llm_processor.save_to_excel([result], "output_file_v2.xlsx", unique_index)

In [154]:
df_parties = pd.read_excel("output_file_v2.xlsx", sheet_name="parties")
df_arguments = pd.read_excel("output_file_v2.xlsx", sheet_name="arguments")

In [155]:
print(f"Shape of parties df: {df_parties.shape}")
Markdown(f"{df_parties.to_markdown(index=False)}")

Shape of parties df: (10, 4)


| person                         | role                 | background                                                                                                                                                                                                                                                      |   unique_index |
|:-------------------------------|:---------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------:|
| Mr. Keifer                     | Defendant            | Resident of Illinois, involved in a legal dispute with Travelers Commercial Insurance regarding the removal of a state court citation to discover assets. An agreement was made with the plaintiffs for a judgment and a stipulation not to execute in Florida. |            249 |
| Travelers Commercial Insurance | Defendant/Appellee   | Insurance company from Connecticut, involved in a legal dispute over the removal of a state court citation to discover assets and a subsequent answer under oath stating no property or monies belonging to the judgment debtor.                                |            249 |
| Mr. Bustamante                 | Plaintiff's Counsel  | Representing the plaintiffs, arguing that the removal was improper and there was no independent controversy at the time of removal. Contends that the case should not have been removed and that the removal violated the forum defendant rule.                 |            249 |
| Mr. Valentino                  | Defendant's Counsel  | Representing Travelers Commercial Insurance, arguing for the affirmation of the district court's decisions and maintaining that the removal of the state court citation to discover assets was appropriate.                                                     |            249 |
| Margaret Esmechbriah           | Justice              | Presiding over the case National Fire Insurance Company versus Lewis Sanchez.                                                                                                                                                                                   |            282 |
| Ellis                          | Justice              | nan                                                                                                                                                                                                                                                             |            282 |
| Cops                           | Justice              | nan                                                                                                                                                                                                                                                             |            282 |
| Brad Stavis                    | Appellant's Attorney | Represents the appellant Mr. Sanchez.                                                                                                                                                                                                                           |            282 |
| Todd Shank                     | Appellee's Attorney  | On behalf of Appellee East Continental Insurance Company and National Fire Insurance Company of Hartford.                                                                                                                                                       |            282 |
| Lewis Sanchez                  | Appellant            | Represented a class of individuals whose biometric information was collected and used by Visual Pack and Elite Staffing.                                                                                                                                        |            282 |

In [156]:
print(f"Shape of arguments df: {df_arguments.shape}")
Markdown(f"{df_arguments.to_markdown(index=False)}")

Shape of arguments df: (10, 5)


| topic                                           | side      | person         | summary                                                                                                                                                                                                                                                                                                                                                            |   unique_index |
|:------------------------------------------------|:----------|:---------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------:|
| Improper Removal                                | Plaintiff | Mr. Bustamante | Argues that the removal of the case was improper as there was no independent controversy to adjudicate at the time of removal, and it violated the forum defendant rule.                                                                                                                                                                                           |            249 |
| No Independent Controversy                      | Plaintiff | Mr. Bustamante | Contends that there was no actual independent controversy issue at the time of removal since Travelers had not filed an answer to the citation, preventing the district court from determining if there was an independent controversy.                                                                                                                            |            249 |
| Answer Under Oath                               | Plaintiff | Mr. Bustamante | Points out that Travelers' answer under oath stated it had no property or monies belonging to the judgment debtor, which should have ended the citation proceeding.                                                                                                                                                                                                |            249 |
| Forum Defendant Rule                            | Plaintiff | Mr. Bustamante | Asserts that removal was also improper under the forum defendant rule because Mr. Keifer, a defendant in the case, is a resident of Illinois.                                                                                                                                                                                                                      |            249 |
| Defense of Proper Removal                       | Defense   | Mr. Valentino  | Defends the removal of the case, stating that the district court correctly applied relevant decisions and that the citation to discover assets presents a separate action requiring its own basis for federal jurisdiction.                                                                                                                                        |            249 |
| Summary Judgment Appropriateness                | Defense   | Mr. Valentino  | Argues that it was appropriate for the judge to rule by summary judgment in the citation proceeding and that the plaintiffs failed to establish a factual basis for a bad faith claim against Travelers.                                                                                                                                                           |            249 |
| No Bad Faith Claim                              | Defense   | Mr. Valentino  | Maintains that the plaintiffs cannot establish a bad faith claim against Travelers as they failed to satisfy the necessary elements and did not provide evidence of Travelers acting in bad faith.                                                                                                                                                                 |            249 |
| Application of Insurance Policy to BIPA Claim   | Appellant | Brad Stavis    | The case involves the application of an insurance policy to a claim brought under the Biometric Information Privacy Act (BIPA). The main issues are whether estoppel applies, preventing the insurance company from asserting exclusions in the policy, and whether there is coverage under the policy based on the West Bend versus Krzysztof Supreme Court case. |            282 |
| Estoppel and Exclusions in Insurance Policy     | Appellant | Brad Stavis    | Argues that estoppel applies due to the insurance company's delay in action, which should prevent them from asserting any exclusions. Even if estoppel does not apply, the violation of law exclusion, which is central to the case, is argued to be ambiguous and should not prevent coverage.                                                                    |            282 |
| BIPA Violations and Insurance Policy Exclusions | Appellee  | Todd Shank     | Claims that estoppel cannot apply unless there is a breach of duty to defend, which is not the case here due to clear exclusions that apply to the BIPA violations in the Sanchez lawsuit. Argues against the ambiguity of the catch-all in the violation of law exclusion and supports the trial court's judgment in favor of CNA.                                |            282 |

In [61]:
document_path = "MariaJimenezvTravelersCommercialInsurance_249.txt"

loader = TextLoader(document_path)
doc = loader.load()

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [62]:
result = chain.invoke("Generate a well strucured list of legal arguments.")

In [63]:
Markdown(f"{result}")

Based on the provided context, here is a well-structured list of legal arguments that appear to be relevant to the case:

1. **Breach of Contract Argument:**
   - The argument that there is no breach of contract by Southwest because the contract allows for various ways Southwest can perform under the agreement.

2. **Definition of Loss Argument:**
   - The costs incurred that are covered by the first definition of loss are not limited to liabilities but also include voluntary decisions made by the policyholder as a result of the system failure.

3. **Excessive Costs Argument:**
   - The carrier has the right to argue that certain costs were excessive and not solely motivated by the system failure, and this can be presented to the fact-finder.

4. **Historical Practice Argument:**
   - Southwest's actions were consistent with its historical practices, and the benefits provided were in line with what it has historically used.

5. **Comparative Value Argument:**
   - The per passenger value of the benefits provided by Southwest was less than that provided by Delta in a similar situation, suggesting that the costs were reasonable.

6. **Voluntary Decision Argument:**
   - The policy covers voluntary decisions to incur negative consequences, as indicated by the etymology of the word "incur."

7. **Irrelevance of the Seventh Circuit Decision Argument:**
   - The Seventh Circuit decision is immaterial to the case at hand as it dealt with a passenger's contract claim, not the insurance policy coverage.

8. **Consequential Damages Argument:**
   - The policy is presumed to exclude consequential damages that are not directly related to the covered occurrence.

9. **First Impression Argument:**
   - This case is of first impression, meaning there is no existing precedent directly on point, and the court's decision will set a new precedent.

10. **Mitigation Costs Argument:**
    - Costs incurred by a business to mitigate further losses due to a covered occurrence should be considered a covered loss under the policy.

11. **Contractual Obligation Argument:**
    - If certain actions are required by the contract of carriage (e.g., taking care of passengers affected by a system failure), they may be excluded from coverage under the liability to third parties exclusion.

12. **Policy Exclusions Argument:**
    - Exclusions in the policy serve to narrow down coverage and should be interpreted to subtract from coverage, not to render the policy illusory.

13. **Policy Interpretation Argument:**
    - The plain language of the policy should guide the interpretation, and the court should adhere to the clear meaning of the terms used in the policy.

These arguments reflect the various positions and interpretations of the insurance policy and contractual obligations at issue in the case.

In [64]:
result_summary = chain.invoke(f"Based on the following list of arguments, distill each side's most important points:\n\n{result}")

In [65]:
Markdown(f"{result_summary}")

Based on the provided context, here are the most important points distilled from each side's arguments:

**Southwest's Side:**

1. **Breach of Contract Argument:**
   - Southwest contends that there is no breach of contract as the contract allows for various performance methods.

2. **Definition of Loss Argument:**
   - Southwest argues that the definition of loss includes both liabilities and voluntary decisions made due to system failure.

3. **Historical Practice Argument:**
   - Southwest maintains that its mitigation actions were consistent with historical practices, suggesting they were reasonable and should be covered.

4. **Voluntary Decision Argument:**
   - The policy is argued to cover voluntary decisions to incur negative consequences, which is consistent with the etymology and common understanding of "incur."

5. **Irrelevance of the Seventh Circuit Decision Argument:**
   - Southwest asserts that the Seventh Circuit decision is not relevant to the current case, which focuses on insurance policy coverage.

6. **Mitigation Costs Argument:**
   - Southwest believes that costs incurred to mitigate further losses due to a covered occurrence are covered losses under the policy.

**Insurance Carrier's Side:**

1. **Consequential Damages Argument:**
   - The carrier argues that the policy excludes consequential damages that are not directly related to the covered occurrence.

2. **First Impression Argument:**
   - The carrier notes that the case is of first impression and will set a new precedent, implying the need for careful consideration of the policy's terms.

3. **Contractual Obligation Argument:**
   - The carrier suggests that actions required by the contract of carriage may be excluded from coverage under the liability to third parties exclusion.

4. **Policy Exclusions Argument:**
   - The carrier emphasizes that exclusions in the policy are meant to narrow coverage and should not be interpreted to render the policy illusory.

5. **Policy Interpretation Argument:**
   - The carrier insists on a plain language interpretation of the policy, adhering to the clear meaning of the terms used.

Both sides present arguments related to the interpretation of the insurance policy, the definition of loss, and the relevance of contractual obligations and historical practices. The court's decision will hinge on the interpretation of these arguments and how they apply to the policy in question.

In [66]:
plaintiff_summary = chain.invoke("Craft a summary of the plaintiff's most key arguments in logical step-by-step fashion.")

In [67]:
Markdown(f"{plaintiff_summary}")

The plaintiff's key arguments can be summarized in a logical step-by-step fashion as follows:

1. **Issue of Sole Causation**: The plaintiff argues that the central question of the case is whether expenses incurred voluntarily by a policyholder to mitigate the impact of a covered business interruption can be considered as incurred solely as a result of the interruption. The plaintiff contends that this is typically a factual matter to be determined by a jury.

2. **Summary Judgment Challenge**: The plaintiff challenges the district court's summary judgment, which was based on the premise that voluntary mitigation expenses cannot be solely caused by a system failure due to the policyholder's business judgment being a concurrent cause. The plaintiff asserts that this judgment is erroneous.

3. **Policy Text and Precedent**: The plaintiff suggests that the case should be decided by looking at the policy text and by drawing analogies to prior court precedents, particularly when there is no existing case law directly addressing the policy language in question.

4. **Analogous Case - Wells vs. Minnesota Life Insurance**: The plaintiff cites the decision in Wells vs. Minnesota Life Insurance, where the court held that subsequent events stemming from an accidental injury do not necessarily sever the sole cause link for policy coverage. The plaintiff draws a parallel between this case and the current situation, arguing that the expenses were a direct result of the system failure.

5. **Evidence of Sole Causation**: The plaintiff points to evidence, including testimony from decision-makers at Southwest, affirming that the compensation decisions were made solely because of the system failure. This evidence is presented to support the argument that the expenses should be covered by the insurance policy.

6. **Nexus to System Failure**: The plaintiff addresses the concern about the nexus between the system failure and the compensation of individuals who did not travel during the interruption. The argument is that the compensation was linked to individual booking numbers, which could include up to eight passengers, thus establishing a connection to the system failure.

7. **Fact Finder's Role**: The plaintiff argues that it is the role of the fact finder to determine whether the compensation was excessive or beyond what was necessary due to the system failure. The plaintiff suggests that the insurer has the right to argue this point before the fact finder.

8. **Historical Business Practices**: The plaintiff asserts that the compensation provided was consistent with Southwest's historical practices, which is relevant to the policy's consideration of the insured's prior business experience.

9. **Definition of "Incurs"**: The plaintiff emphasizes the meaning of the term "incurs," arguing that it refers to a loss that an individual or entity brings upon itself, often voluntarily. This interpretation supports the plaintiff's view that the policy covers the expenses in question.

10. **Irrelevance of the Seven Circuit Decision**: Finally, the plaintiff dismisses a Seven Circuit decision cited by the opposition as immaterial, stating that it pertained to a passenger's contract claim against Southwest and not to the insurance policy coverage issue at hand.

The plaintiff concludes by requesting that the summary judgment be reversed, allowing the matter to be decided by a fact finder, as the evidence suggests that the expenses were indeed incurred solely as a result of the system failure.

In [70]:
defense_summary = chain.invoke("Using the following arguments, craft a concise, methodical, fact-focused counter argument as a bullet point list. Look for errors and omissions in the plaintiff logic:\n\n{plaintiff_summary}")
Markdown(f"{defense_summary}")

Counter Argument:

- **Voluntary Mitigation vs. Sole Causation:**
  - The plaintiff conflates voluntary mitigation efforts with expenses solely caused by the system failure. Voluntary actions may have multiple motivations beyond the immediate cause.
  - The policy language likely distinguishes between actions taken solely due to the interruption and those that are part of broader business strategies, including goodwill gestures.

- **Historical Practices Not Necessarily Relevant:**
  - Past practices of offering benefits do not automatically justify current expenses as being solely caused by the system failure.
  - The historical context provided by the plaintiff does not address whether the scale or scope of the mitigation efforts was proportionate to the interruption.

- **Excessive Compensation Claims:**
  - The plaintiff admits that the carrier could argue against excessive compensation, suggesting that there is a limit to what can be considered solely caused by the interruption.
  - The example of lifetime travel for goodwill exceeds reasonable mitigation and indicates that there is a line where voluntary decisions are no longer solely due to the system failure.

- **Fair Saver Program for Friends:**
  - The inclusion of friends in the compensation package lacks a direct nexus to the system failure, as they were not affected passengers.
  - The plaintiff's argument does not adequately address why non-traveling individuals should be compensated for an interruption they did not experience.

- **Advertising Costs:**
  - The plaintiff does not justify why a full week of advertising costs would be covered when only a day was lost, suggesting a lack of proportionality in the mitigation expenses.

- **Seven Circuit Decision:**
  - The plaintiff dismisses the Seven Circuit decision as immaterial, but it may provide insight into how contractual obligations are interpreted in relation to system failures and subsequent actions.

- **Etymology of "Incur":**
  - The plaintiff's reliance on the etymology of "incur" is not a strong legal argument. The policy terms and conditions, not the Latin root of words, should determine coverage.

- **Lack of Precedent:**
  - The plaintiff acknowledges the absence of precedent in the business interruption context, which suggests that the court should be cautious in extending coverage to voluntary mitigation expenses without clear guidance.

- **Fact Finder's Role:**
  - The plaintiff's argument that all issues should go to the fact finder overlooks the possibility that some issues may be decided as a matter of law if the policy language is clear.

- **Policy Interpretation:**
  - The plaintiff's interpretation of the policy may be overly broad, potentially leading to coverage for actions that extend beyond the intent of the policy and the reasonable expectations of the insurer.

In [71]:
system_failure = chain.invoke("Generate a detailed and comprehensive summary of the 'system failure' being argued. Include facts and quotes as needed to.")
Markdown(f"{system_failure}")

The discussion revolves around a 'system failure' experienced by Southwest Airlines and the subsequent insurance coverage dispute. The system failure led to significant disruptions, including the stranding of approximately 400,000 passengers. Southwest made a series of decisions to mitigate the impact on customers, which included offering refunds, vouchers, and other gestures of goodwill. These decisions were not legally mandated but were made unilaterally by Southwest as a business judgment to maintain customer relations and goodwill.

The crux of the argument is whether these voluntary, goodwill measures taken by Southwest in response to the system failure are covered under their business interruption insurance policy. The policy specifies coverage for losses "solely as a result of the system failure," which has become a point of contention. Southwest argues that all the customer compensation decisions were undertaken solely because of the system failure, suggesting that these should be covered by the insurance.

The insurer, on the other hand, contends that the measures go beyond what is covered by the policy. They argue that the policy is intended to put the insured in the position they would have been in if the system failure had not occurred, and does not cover the downstream effects such as PR or customer goodwill measures. The insurer maintains that the policy does not cover costs that are not a direct necessity or that go beyond restoring the company to the position it was in before the system failure.

The discussion also touches on the nature of the benefits provided to customers, including the 'Fair Saver' promo codes, which were linked to individual booking numbers and could cover up to eight passengers. The insurer challenges the nexus between these benefits and the system failure, particularly for passengers who did not travel during the disruption. Southwest defends this by stating that the actual usage of the benefits resulted in an average of 1.5 tickets per promo code, and that these measures were consistent with their historical practices in similar situations.

The debate also includes a legal analysis of what constitutes a loss that is "incurred" under the policy, with Southwest's counsel pointing to the etymology of the word "incur" to argue that it implies a self-inflicted negative consequence, which should be covered by the policy.

The court is tasked with determining whether the voluntary measures taken by Southwest are indeed covered losses under the insurance policy, considering the policy's language and the intent behind the business interruption coverage. The insurer suggests that the court's role is to differentiate between out-of-pocket costs directly associated with the system failure and benefits that may extend beyond that scope. The outcome of this dispute will hinge on the interpretation of the policy language and the factual circumstances surrounding the system failure and Southwest's response.